# First We Install Some Libraries
The basic library if transformrers.  We also need accelerate for some reaon.

In [2]:
!pip install --upgrade pip
!pip install transformers 
!pip install accelerate 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
model = T5ForConditionalGeneration.from_pretrained("google/flan-ul2", torch_dtype=torch.bfloat16, device_map="auto")     
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

This is a function that, given a query, generates a response from the LLM.  

In [2]:
def f(str):
    inputs = tokenizer(str, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(inputs, max_length=500,
                             num_beams=2,
                             repetition_penalty=2.5,
                             length_penalty=1.0,
                             early_stopping=True,
                             no_repeat_ngram_size=2,
                             use_cache=True,
                             do_sample = True,
                             temperature = 1.5,
                             top_k = 50,
                             top_p = 0.95)
    
    print(tokenizer.decode(outputs[0][1:-1]))

We now test if the model works.  This is tuned to answer questions with short responses, so it gives just a few words as the answer.  We use a pretty high temperature so the output often changes.  

Sometimes it says *record sales* or *singer songwriter* or *he had long and bizarre hair* which I suppose are arguably correct.

In [3]:
f("How did David Bowie become famous?")

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


songwriter and musician


Now we install peft, the package that allows parameter efficient fine tuning.  We will also need datasets (to load a dataset).

In [4]:

!pip install "peft==0.2.0" --quiet
!pip install "transformers==4.27.2" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" loralib  --quiet
!pip install "datasets==2.9.0"

!pip install protobuf=="3.20.*" --quiet

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import json
jjout = []
with open("/home/ubuntu/five-dollar-test/train.json") as file:
    while True:
        line = file.readline()
        if not line:
            break
        i = json.loads(line)
        if not i.get("input") or not i.get("output"):
            continue
        jjout.append({"input":  i["input"], "output": i["output"]})
        
import random
random.shuffle(jjout)
import os
try:
    os.mkdir("/home/ubuntu/data/")
except:
    print("Directory /home/ubuntu/data/ exists")

with open("/home/ubuntu/data/train.json", "w") as file:
    file.write(json.dumps(jjout[100:]))

from datasets import load_dataset
data_files = {"train": "train.json"}
dataset = load_dataset("/home/ubuntu/data/", data_files=data_files)
save_dataset_path = "/home/ubuntu/data-all-test" 

max_length = 512 #tokenizer.model_max_length

def preprocess_function(sample, padding="max_length"):
    inputs = [f"{{input}}\n".format(input=item) for item in sample['input']]
    model_inputs = tokenizer(inputs, max_length=max_length, padding=padding, truncation=True)
    labels = tokenizer(text_target=sample['output'], max_length=max_length, padding=padding, truncation=True)
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=list(dataset["train"].features))
tokenized_dataset["train"].save_to_disk(os.path.join(save_dataset_path,"train"))


Directory /home/ubuntu/data/ exists


We load our dataset from disk.

In [5]:
from datasets import load_from_disk
tokenized_dataset = load_from_disk("/home/ubuntu/data-all-test/train")

Now we make the lora model which is a wrapper around the base model.

In [6]:
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)

import torch.nn as nn
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


model.print_trainable_parameters()

trainable params: 25165824 || all params: 19484779520 || trainable%: 0.12915631903439675


In [7]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [8]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="lora-flan-ul2-fact"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    #auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=1,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=50,
    max_steps=1000, # takes 
    bf16=True,
    save_strategy="no",
    report_to="tensorboard",
    optim='adamw_torch'
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)
model.config.use_cache = False 

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,2.300200


In [12]:
1+1

2

This saves the peft model in results-ul2-1k.  Then it merges the peft model into the UL2 model to make a new model and saves it as merged-ul2-model

In [14]:
peft_model_id="results-ul2-1k"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

key_list = [key for key, _ in model.base_model.model.named_modules() if "lora" not in key]
for key in key_list:
    parent, target, target_name = model.base_model._get_submodules(key)
    if isinstance(target, peft.tuners.lora.Linear):
        bias = target.bias is not None
        new_module = torch.nn.Linear(target.in_features, target.out_features, bias=bias)
        model.base_model._replace_module(parent, target_name, new_module, target)

model = model.base_model.model
model.save_pretrained("/home/ubuntu/merged-ul2-model")



('results-fact-ul2-1k-v2/tokenizer_config.json',
 'results-fact-ul2-1k-v2/special_tokens_map.json',
 'results-fact-ul2-1k-v2/tokenizer.json')

This is how to load the peft model.  It is faster to load the merged model however.

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer



# Load peft config for pre-trained checkpoint etc.
peft_model_id = "results-f-ul2-1k-v2"
config = PeftConfig.from_pretrained(peft_model_id)
print("config")
# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  device_map={"":0})
print("model")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
print("tokenizer")
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
print("lora")
model.eval()
print("eval")
print("Peft model loaded")

def f(sample):
    input_ids = tokenizer(sample, return_tensors="pt", truncation=True).input_ids.cuda()
    # with torch.inference_mode():
    outputs = model.generate(input_ids=input_ids, 
                             max_new_tokens=500, 
                             do_sample=True, 
                             top_p=0.9,
                             num_beams=3,
                             repetition_penalty=2.5,
                             early_stopping=True,
                             no_repeat_ngram_size=2,
                             use_cache=True,
                             top_k = 50)
    print(f"input sentence: {sample}\n{'---'* 10}")
    print(f"Answer:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

f("does the white house have a dining room")


config


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

model
tokenizer
lora
eval
Peft model loaded


/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


input sentence: does the white house have a dining room
------------------------------
Answer:
The White House dining room is one of the most famous rooms in Washington, D.C. It's where the president and first lady dine on a daily basis, and it has been home to many historic events since its construction in 1792. In addition to hosting dignitaries from around the world, the dining area also serves as an important ceremonial space for official dinners and luncheons. A few years ago, we took you behind the scenes of this iconic space with our "Behind the Scenes" series. Now, let us take you through the history of how the Dining Room became what it is today. As the story goes, President George Washington wanted his guests to be able to enjoy their meals without having to leave the building. He decided to build the first presidential dining hall at the north end of Pennsylvania Avenue, across from the U.S. Capitol Building. The design was inspired by Italian Renaissance palaces, including 

In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer =AutoTokenizer.from_pretrained("google/flan-ul2")


model = AutoModelForSeq2SeqLM.from_pretrained("/home/ubuntu/merged-ul2-model",torch_dtype=torch.bfloat16, device_map="auto")

def f(sample):
    input_ids = tokenizer(sample, return_tensors="pt", truncation=True).input_ids.cuda()
    # with torch.inference_mode():
    outputs = model.generate(input_ids=input_ids, 
                             max_new_tokens=500, 
                             do_sample=True, 
                             top_p=0.9,
                             num_beams=3,
                             repetition_penalty=2.5,
                             early_stopping=True,
                             no_repeat_ngram_size=2,
                             use_cache=True,
                             top_k = 50)
    print(f"input sentence: {sample}\n{'---'* 10}")
    print(f"Answer:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

f("does the white house have a dining room")


input sentence: Does the white house have a pool
------------------------------
Answer:
The White House has a pool, but it's not exactly what you'd call an "infinity pool." In fact, the pool isn't even in the house itself. Instead, it sits on the grounds of the Washington Monument and is accessible to the public through the National Park Service. While some people might be surprised by the location of this swimming hole, others will be familiar with its history. The pool was originally built in 1812 as part of President Thomas Jefferson'


In [5]:
f("what can newborns see")

input sentence: what can newborns see
------------------------------
Answer:
During infancy, infants have limited vision. Their eyes are closed and they can only see black and white images. When their eyes open at around four months of age, they begin to see color. However, it is not until the first year of life that they will fully develop their ability to distinguish colors. In fact, newborns may be able to recognize red and green but not blue or purple. A baby's sense of sight begins developing shortly after birth. The eye has two parts: the retina (the light-sensitive layer) and the optic nerve (which connects the eye to the brain). The retina converts light into electrical impulses that are sent to your brain. This process is called photoreception. Your brain then interprets these signals as an image. There are three stages of visual development during childhood: prenatal, early childhood, and late childhood. Prenatal development occurs before a baby is born. It includes the forma

In [8]:
f("how can the ukraine win the war")

input sentence: how can the ukraine win the war
------------------------------
Answer:
The war in eastern Ukraine has been going on for over a year now, and both sides have suffered significant casualties. While the Ukrainian government claims to be winning, it is not clear that this is the case. According to the United Nations High Commissioner for Refugees (UNHCR), as of December 1, 2018, more than 11,000 people had been killed in the conflict, with an additional 33,000 wounded. In addition, there are nearly 200,000 internally displaced persons (IDPs) who have been forced to leave their homes due to violence or lack of humanitarian aid. A recent report by the UN Special Rapporteur on the situation of IDPs in Ukraine, Nadine El-Baghdadi, concluded that "the number of civilian deaths and injuries will continue to rise" unless immediate action is taken to address the crisis.
